# Using Spectral Indices for Drainage Pipe Detection

This notebook demonstrates how to use spectral indices to enhance drainage pipe detection with the DrainageAI system. Spectral indices like NDVI, NDMI, and MSAVI2 can highlight patterns in the landscape that are indicative of subsurface drainage systems.

## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import rasterio
from rasterio.plot import show

# Add parent directory to path for imports
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# Import DrainageAI modules
from models import CNNModel
from preprocessing import ImageProcessor

## 1. Calculate Spectral Indices

Let's define functions to calculate the most useful spectral indices for drainage pipe detection.

In [ ]:
def calculate_ndvi(red, nir):
    """Calculate Normalized Difference Vegetation Index."""
    return (nir - red) / (nir + red + 1e-8)  # Add small epsilon to avoid division by zero

def calculate_ndmi(nir, swir):
    """Calculate Normalized Difference Moisture Index."""
    return (nir - swir) / (nir + swir + 1e-8)

def calculate_msavi2(red, nir):
    """Calculate Modified Soil Adjusted Vegetation Index 2."""
    return (2 * nir + 1 - np.sqrt((2 * nir + 1)**2 - 8 * (nir - red))) / 2

def calculate_ndwi(green, nir):
    """Calculate Normalized Difference Water Index."""
    return (green - nir) / (green + nir + 1e-8)

def calculate_savi(red, nir, L=0.5):
    """Calculate Soil Adjusted Vegetation Index."""
    return (nir - red) * (1 + L) / (nir + red + L + 1e-8)

Now, let's load a sample multispectral image and calculate the indices.

In [ ]:
# Load sample multispectral image
# Replace with your own data path
image_path = "../data/sample_imagery.tif"

# For demonstration, we'll create a synthetic image if the file doesn't exist
if not os.path.exists(image_path):
    print("Sample imagery not found. Creating synthetic data for demonstration...")
    # Create a directory for the data if it doesn't exist
    os.makedirs(os.path.dirname(image_path), exist_ok=True)
    
    # Create synthetic multispectral image (5 bands: Blue, Green, Red, NIR, SWIR)
    height, width = 500, 500
    synthetic_image = np.random.rand(5, height, width).astype(np.float32)
    
    # Add some patterns that might look like drainage pipes
    # Horizontal lines
    for i in range(50, height, 100):
        synthetic_image[3, i:i+3, :] = 0.8  # Higher NIR reflectance along lines
        synthetic_image[0, i:i+3, :] = 0.2  # Lower blue reflectance along lines
    
    # Vertical lines
    for j in range(50, width, 100):
        synthetic_image[3, :, j:j+3] = 0.8  # Higher NIR reflectance along lines
        synthetic_image[0, :, j:j+3] = 0.2  # Lower blue reflectance along lines
    
    # Save synthetic image
    with rasterio.open(
        image_path,
        'w',
        driver='GTiff',
        height=height,
        width=width,
        count=5,
        dtype=synthetic_image.dtype,
        crs='+proj=utm +zone=17 +datum=WGS84',
        transform=rasterio.transform.from_bounds(0, 0, width, height, width, height)
    ) as dst:
        dst.write(synthetic_image)
    
    print(f"Synthetic image created and saved to {image_path}")

# Load the image
with rasterio.open(image_path) as src:
    image = src.read()
    meta = src.meta
    
    # Extract bands (assuming standard order: Blue, Green, Red, NIR, SWIR)
    blue = image[0]
    green = image[1]
    red = image[2]
    nir = image[3]
    swir = image[4] if image.shape[0] > 4 else None

# Calculate indices
ndvi = calculate_ndvi(red, nir)
msavi2 = calculate_msavi2(red, nir)
ndwi = calculate_ndwi(green, nir)
savi = calculate_savi(red, nir)

# Calculate NDMI if SWIR band is available
ndmi = calculate_ndmi(nir, swir) if swir is not None else None

# Display the original image (RGB composite)
plt.figure(figsize=(10, 8))
plt.imshow(np.dstack((red, green, blue)))
plt.title("RGB Composite")
plt.axis('off')
plt.show()

# Display the calculated indices
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].imshow(ndvi, cmap='RdYlGn')
axes[0, 0].set_title("NDVI")
axes[0, 0].axis('off')

axes[0, 1].imshow(msavi2, cmap='RdYlGn')
axes[0, 1].set_title("MSAVI2")
axes[0, 1].axis('off')

axes[1, 0].imshow(ndwi, cmap='Blues')
axes[1, 0].set_title("NDWI")
axes[1, 0].axis('off')

if ndmi is not None:
    axes[1, 1].imshow(ndmi, cmap='Blues')
    axes[1, 1].set_title("NDMI")
else:
    axes[1, 1].imshow(savi, cmap='RdYlGn')
    axes[1, 1].set_title("SAVI")
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 2. Modify CNN Model to Use Spectral Indices

Now, let's modify the CNN model to incorporate spectral indices as additional input channels.

In [ ]:
class SpectralCNNModel(CNNModel):
    """CNN model that incorporates spectral indices as additional input channels."""
    
    def __init__(self, num_indices=3, pretrained=True):
        super().__init__(pretrained=pretrained)
        
        # Replace first conv layer to accept additional channels for indices
        original_conv = self.features[0]
        self.features[0] = torch.nn.Conv2d(
            3 + num_indices,  # RGB + spectral indices
            original_conv.out_channels,
            kernel_size=original_conv.kernel_size,
            stride=original_conv.stride,
            padding=original_conv.padding,
            bias=original_conv.bias is not None
        )
        
        # Initialize the new conv layer
        if pretrained:
            # Copy weights for RGB channels from pre-trained model
            with torch.no_grad():
                self.features[0].weight[:, :3] = original_conv.weight
                # Initialize weights for spectral indices channels
                torch.nn.init.kaiming_normal_(
                    self.features[0].weight[:, 3:],
                    mode='fan_out',
                    nonlinearity='relu'
                )
                if original_conv.bias is not None:
                    self.features[0].bias = original_conv.bias
    
    def forward(self, x):
        """Forward pass of the model."""
        # Handle different input formats
        if isinstance(x, dict):
            # Combine RGB imagery with spectral indices
            imagery = x['imagery']
            indices = x['indices']
            
            # Ensure indices have the right shape
            if indices.shape[2:] != imagery.shape[2:]:
                indices = torch.nn.functional.interpolate(indices, size=imagery.shape[2:], mode='bilinear')
            
            # Concatenate along channel dimension
            x = torch.cat([imagery, indices], dim=1)
        
        # Continue with normal forward pass
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
    def preprocess(self, data):
        """Preprocess input data before feeding to the model."""
        # If data is already a dictionary with 'imagery' and 'indices', return as is
        if isinstance(data, dict) and 'imagery' in data and 'indices' in data:
            return data
        
        # Otherwise, assume data is just imagery and preprocess it
        return super().preprocess(data)

## 3. Prepare Input Data with Spectral Indices

Now, let's prepare the input data for our model, including both the original imagery and the calculated spectral indices.

In [ ]:
# Prepare input data
# Normalize the original image
image_processor = ImageProcessor()
normalized_image = image_processor.preprocess(image[:3])  # Use only RGB bands for simplicity

# Stack the selected indices
indices_stack = np.stack([ndvi, msavi2, ndmi if ndmi is not None else savi])

# Convert to PyTorch tensors
imagery_tensor = torch.from_numpy(normalized_image).float()
indices_tensor = torch.from_numpy(indices_stack).float()

# Create input dictionary
input_data = {
    'imagery': imagery_tensor.unsqueeze(0),  # Add batch dimension
    'indices': indices_tensor.unsqueeze(0)   # Add batch dimension
}

print(f"Imagery tensor shape: {input_data['imagery'].shape}")
print(f"Indices tensor shape: {input_data['indices'].shape}")

## 4. Run Inference with the Modified Model

Now, let's create an instance of our modified CNN model and run inference on the prepared data.

In [ ]:
# Create model
model = SpectralCNNModel(num_indices=3, pretrained=True)
model.eval()

# Run inference
with torch.no_grad():
    output = model(input_data)
    
    # Convert output to binary mask
    binary_mask = (output > 0.5).float()

# Convert to numpy for visualization
output_np = output.squeeze().numpy()
binary_mask_np = binary_mask.squeeze().numpy()

# Display results
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(output_np, cmap='jet')
axes[0].set_title("Probability Map")
axes[0].axis('off')

axes[1].imshow(binary_mask_np, cmap='gray')
axes[1].set_title("Binary Mask")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 5. Visualize Results with Original Imagery

Finally, let's overlay the detection results on the original imagery for better visualization.

In [ ]:
# Create RGB composite
rgb = np.dstack((red, green, blue))

# Normalize RGB for display
rgb_norm = (rgb - rgb.min()) / (rgb.max() - rgb.min())

# Create overlay
overlay = rgb_norm.copy()
overlay[binary_mask_np > 0.5] = [1, 0, 0]  # Red color for detected pipes

# Display original and overlay
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(rgb_norm)
axes[0].set_title("Original RGB")
axes[0].axis('off')

axes[1].imshow(overlay)
axes[1].set_title("Drainage Pipe Detection")
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 6. Save Results

Let's save the detection results as a GeoTIFF file that can be opened in ArcGIS or QGIS.

In [ ]:
# Save detection results
output_path = "../data/drainage_detection_result.tif"

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Save as GeoTIFF
with rasterio.open(
    output_path,
    'w',
    driver='GTiff',
    height=binary_mask_np.shape[0],
    width=binary_mask_np.shape[1],
    count=1,
    dtype=binary_mask_np.dtype,
    crs=meta['crs'],
    transform=meta['transform']
) as dst:
    dst.write(binary_mask_np, 1)

print(f"Detection results saved to {output_path}")

## Conclusion

In this notebook, we've demonstrated how to:

1. Calculate spectral indices (NDVI, NDMI, MSAVI2, etc.) from multispectral imagery
2. Modify the CNN model to incorporate these indices as additional input channels
3. Prepare input data with both imagery and indices
4. Run inference with the modified model
5. Visualize and save the results

This approach can significantly improve drainage pipe detection accuracy, especially in areas with limited labeled data. The spectral indices provide valuable information about vegetation health, soil moisture, and other factors that can indicate the presence of subsurface drainage systems.